# Notebook for developing code to simulate the SiPM pulses

In [1]:
import os
import numpy as np
import pandas as pd

In [14]:
class Particle:
    """ """
    def __init__(self, type: str = 'MIP', pos: np.array = np.zeros(4), velocity: np.array = np.zeros(3)):
        self.type     = type
        self.pos      = pos
        self.velocity = velocity
        self.energy   = None
    
    
    
# class MIP(Particle):
    
    
# class Photon(Particle):
    

In [16]:
myparticle = Particle()
myparticle.type

AttributeError: can't set attribute

In [ ]:
class ParticleGun:
    

In [ ]:
class Scintillator():
    
    def __init__():
        
    def process(particle: Particle = Particle()):
        
        return photons

In [ ]:
class SiPM():
    

In [ ]:
class Electronics():
    

In [ ]:
class Digitizer():

In [94]:
class Event():

    def __init__(self, idx: int = -1):
        self.idx = idx
        self.particles = []
        self.photons = []
        self.sipm_waveform = None
        self.electronics_waveform = None
        self.digitized_waveform = None
    
    def get_meta(self):
        
        columns = ['particle_file',
                   'photon_file',
                   'digitized_wf_file',
                   'n_particles',
                   'n_photons']
        
        data    = [str(self.idx).zfill(7) + '_particles.csv',
                   str(self.idx).zfill(7) + '_photons.csv',
                   str(self.idx).zfill(7) + '_digitzed_wf.csv',
                   np.nan,
                   np.nan]
        
        meta    = pd.Series(data, index = columns) 
        
        return meta
    

In [95]:
str().zfill(7)

'0000000'

In [96]:
class SimulationHandler:
    
    def __init__(self):
        
        self.idx          = -1
        self.particle_gun = None
        self.scintillator = None
        self.sipm         = None 
        self.electronics  = None
        self.digitizer    = None
        
        self.events = []
        self.events_df = pd.DataFrame(columns = ['particle_file',
                                                 'photon_file',
                                                 'digitized_wf_file',
                                                 'n_particles',
                                                 'n_photons'])
    
    def simulate_event(self):
        
        self.idx += 1
        event = Event(self.idx)
   
        self.events.append(event)
        

        
#     def simulate_and_save_event(self):
    """ TODO IMPLEMENT"""
    
    
    def save_events(self, root_dir):
        
        for event in self.events:
            meta = event.get_meta()
            
            # save particle_file
            # save photon_file
            # save waveform_file
            self.events_df = self.events_df.append(meta, ignore_index=True)
            
            
        self.events_df.to_csv(os.path.join(root_dir,'test.csv'))
             

In [97]:
handler = SimulationHandler()

handler.simulate_event()
handler.simulate_event()
handler.save_events(r'C:\Users\Miroslav Gabriel\Workspace\pulseshape_discrimination\data\test')

In [98]:
def modify(number):
    number = number + 1
    
number = 5

modify(number)

print(number)

5
